In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel as fastapiBaseModel#이건 fastapi에서 사용하는 basemodel
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import Field
from langchain_core.pydantic_v1 import BaseModel as LangChainBaseModel#이건 아웃풋 고정할 때 사용하는 basemodel, 둘 다 있어야 함, 둘 다 이름 바꿔줘야 오류 안나니 주의
from langchain.memory import ChatMessageHistory
import os
from langchain_core.output_parsers import StrOutputParser
# from langchain.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import json

In [12]:
load_dotenv(dotenv_path="../.env")


True

In [17]:
name = ""
Gpersona = ""

def load_persona(session_id):#페르소나 로드
    global name
    global Gpersona
    if name==session_id:
        return Gpersona#같은 사람이랑 이야기할 때는 계속 페르소나를 읽지 않고 저장된 변수에서 가져오도록
    name = session_id
    persona_file = f"../Personas/persona_{session_id}.txt"
    if os.path.isfile(persona_file):
        with open(persona_file, 'r', encoding='utf-8') as f:
            persona = f.read()
            Gpersona = persona
    else:
        print(f"파일 '{persona_file}'이 존재하지 않습니다.")
        persona = "" 
    return persona
load_persona('Lily')

해당 장소에서 할 수 있는 활동들 목록 > 해당 장소의 사물 및 애니메이션에 따라 달라짐

In [66]:
house_actions = [
    "Cook a meal",
    "Watch TV",
    "Read a book",
    "Exercise",
    "Sleep",
    "Clean the house",
    "Work from home",
    "Call a friend",
    "Take a shower",
    "Use a desktop"
]
cafe_actions = [
    "Order coffee",
    "Chat with barista",
    "Use free Wi-Fi",
    "Read a magazine",
    "Work on laptop",
    "Meet a friend",
    "Listen to music",
    "People watch",
    "Sketch in a notebook",
    "Try a new pastry"
]
bakery_actions = [
    "Buy bread",
    "Purchase pastries",
    "Order a cake",
    "Chat with baker",
    "Sample free tasters",
    "Watch the baking process",
    "Learn a baking tip",
    "Order coffee",
    "Relax in seating area",
    "Sign up for baking class"
]
park_actions = [
    "Walk the dog",
    "Have a picnic",
    "Jog",
    "Read a book on a bench",
    "Feed the ducks",
    "Play frisbee",
    "Take photos",
    "Ride a bike",
    "Meditate",
    "Fly a kite"
]
convenience_store_actions = [
    "Buy snacks",
    "Purchase drinks",
    "Get lottery tickets",
    "Chat with cashier",
    "Pick up a magazine",
    "Use the ATM",
    "Get a hot dog",
    "Buy household items",
    "Refill phone credit",
    "Get ice cream"
]
bar_actions = [
    "Order a drink",
    "Play pool",
    "Listen to live music",
    "Watch a sports game",
    "Chat with bartender",
    "Dance",
    "Meet new people",
    "Sing karaoke",
    "Try a new cocktail",
    "Participate in trivia night"
]
library_actions = [
    "Borrow a book",
    "Study",
    "Use the computer",
    "Read in a quiet area",
    "Attend a workshop",
    "Join a book club",
    "Listen to an audiobook",
    "Research a topic",
    "Print documents",
    "Donate books"
]


인물의 페르소나에 맞춰 해당 인물의 일정 설계, 일정에는 공간과 무슨 행동을 할 지가 포함됨

In [73]:
persona_ryan = '''
 name : Ryan Park
 age : 29
 Innate tendency : imaginative, patient, kind
 Learned tendency : Tamara Taylor is a children's book author who loves to create stories that capture the imaginations of young readers. She is very patient and kind when it comes to her work.
 Currently : Ryan Park is working on a project to create a new mobile app. He is also reading up on the latest technologies to stay ahead of the curve.
 Lifestyle : Ryan Park goes to bed around 1am, awakes up around 9am, eats dinner around 5pm.
'''
persona_tamara = '''
 name : Tamara Taylor
 age : 30
 Innate tendency : analytical, pragmatic, driven
 Learned tendency : Ryan Park is a software engineer who loves to solve problems. He is constantly looking for ways to improve existing systems.
 Currently : Tamara Taylor lives with a housemate, Carmen Ortiz, and is working on a new series of children's books. She is also working on a comic book series for adults.
 Lifestyle : Tamara Taylor goes to bed around 10pm, awakes up around 6am, eats dinner around 6pm.
'''
persona_abigail = '''
 name : Abigail Chen
 age : 25
 Innate tendency : open-minded, curious, determined
 Learned tendency : Abigail Chen is a digital artist and animator who loves to explore how technology can be used to express ideas. She is always looking for new ways to combine art and technology.
 Currently : Abigail Chen is working on an animation project for a client. She is also experimenting with different tools and techniques to create interactive art.
 Lifestyle : Abigail Chen goes to bed around midnight, awakes up around 8am, eats dinner around 6pm.
'''
#당신은 상상력이 풍부한 게임 스토리 작가입니다. 요청에 답하라.
sysprom = "You are an imaginative game story writer. Answer the request."
#주어진 사람 {페르소나}의 페르소나를 이해합니다. 주변에는 테일러의 집, 카페, 베이커리, 공원, 편의점, 바, 도서관 등이 있습니다. 테일러의 집에서 할 수 있는 일은 {} 입니다. 카페에서 할 수 있는 일은 {} 입니다. 베이커리에서 할 수 있는 일은 {} 입니다. 공원에서 할 수 있는 일은 {} 입니다. 편의점에서 할 수 있는 일은 {} 입니다. 바에서 할 수 있는 일은 {} 입니다. 이해한 정보를 바탕으로 그 사람의 하루 일정을 계획합니다. 계획된 일정을 주어진 파이썬 목록의 형태로 출력합니다. {"hh:mm","place","활동","일정에 대한 자세한 설명"}
userprom = f"""Understand the persona of the given people, {persona_ryan},{persona_abigail},{persona_tamara}.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is {house_actions}. 
What you can do at a cafe is {cafe_actions}. 
What you can do at a bakery is {bakery_actions}. 
What you can do at a park is {park_actions}. 
What you can do at a convenience store is {convenience_store_actions}. 
What you can do at a bar is {bar_actions}. 
Plan each person's daily schedule based on the information you understand. 
Print out the planned schedule in the form of a given list. [['hh:mm','place','activity','Detailed description of the schedule']]"""
userprom = f"""Understand the persona of the given people, {persona_ryan},{persona_abigail},{persona_tamara}.
Around the people, there are places such as Ryan's house, Tamara's house, Abigail's house, cafe, bakery, park, convenience store, bar, and library.
What you can do at Ryan's house, Tamara's house, Abigail's house is {house_actions}. 
What you can do at a cafe is {cafe_actions}. 
What you can do at a bakery is {bakery_actions}. 
What you can do at a park is {park_actions}. 
What you can do at a convenience store is {convenience_store_actions}. 
What you can do at a bar is {bar_actions}. 
Plan each person's daily schedule based on the information you understand. 
Print out the planned schedule in list form in order of name, time, place, action, and explanation."""

In [69]:
userprom

"Understand the persona of the given people, \n name : Ryan Park\n age : 29\n Innate tendency : imaginative, patient, kind\n Learned tendency : Tamara Taylor is a children's book author who loves to create stories that capture the imaginations of young readers. She is very patient and kind when it comes to her work.\n Currently : Ryan Park is working on a project to create a new mobile app. He is also reading up on the latest technologies to stay ahead of the curve.\n Lifestyle : Ryan Park goes to bed around 1am, awakes up around 9am, eats dinner around 5pm.\n,\n name : Abigail Chen\n age : 25\n Innate tendency : open-minded, curious, determined\n Learned tendency : Abigail Chen is a digital artist and animator who loves to explore how technology can be used to express ideas. She is always looking for new ways to combine art and technology.\n Currently : Abigail Chen is working on an animation project for a client. She is also experimenting with different tools and techniques to create

In [74]:
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": sysprom},
        {"role": "user", "content": userprom}
    ],
    temperature=0.5,
    max_tokens=2000
)
print(completion.choices[0].message.content)

### Ryan Park's Daily Schedule

1. **9:00 AM - Home - Wake Up**
   - Ryan starts his day by waking up at 9 AM.

2. **9:30 AM - Home - Take a Shower**
   - Freshens up to kickstart the day.

3. **10:00 AM - Home - Eat Breakfast**
   - Prepares and eats a healthy breakfast.

4. **10:30 AM - Home - Work from Home**
   - Begins working on his mobile app project.

5. **1:00 PM - Home - Call a Friend**
   - Takes a break to catch up with a friend.

6. **1:30 PM - Home - Read a Book**
   - Reads up on the latest technologies to stay ahead of the curve.

7. **2:30 PM - Home - Cook a Meal**
   - Prepares a light lunch.

8. **3:00 PM - Cafe - Order Coffee**
   - Heads to a nearby cafe and orders a coffee.

9. **3:15 PM - Cafe - Work on Laptop**
   - Continues working on his project using the cafe's free Wi-Fi.

10. **5:00 PM - Home - Eat Dinner**
    - Returns home to have dinner.

11. **6:00 PM - Home - Exercise**
    - Does a home workout to stay fit.

12. **7:00 PM - Home - Watch TV**
    - R

In [50]:
completion.choices[0].message.content

"[['06:00', 'Wake up and start the day with a morning stretch and some light exercise.', 'Taylor\\'s house'],\n ['06:30', 'Have a healthy breakfast and a cup of tea while reviewing the day\\'s tasks.', 'Taylor\\'s house'],\n ['07:00', 'Spend some quiet time brainstorming ideas for the new children\\'s book series.', 'Taylor\\'s house'],\n ['08:00', 'Head to the nearby park for a refreshing walk and to gather inspiration from nature.', 'Park'],\n ['09:00', 'Visit the local bakery to pick up a fresh pastry and chat with the friendly staff.', 'Bakery'],\n ['09:30', 'Settle down at the cafe to work on the comic book series for adults, enjoying the ambiance and a cup of coffee.', 'Cafe'],\n ['12:00', 'Take a break for lunch, either at the cafe or back at home with Carmen.', 'Cafe/Taylor\\'s house'],\n ['13:00', 'Return home to continue working on the children\\'s book series, focusing on illustrations and story development.', 'Taylor\\'s house'],\n ['15:00', 'Head to the library to do some 